# Module 3 - Web APIs

# Table of Contents
- [Introduction](#Introduction)

    - [Learning Objectives](#Learning-Objectives)
    - [Topics](#Topics)

- [Setup - Load Python packages](#Setup---Load-Python-packages)
- [Using APIs](#Using-APIs)
    * [RESTful APIs](#RESTful-APIs)
    * [OTP description](#OpenTripPlanner)
    * [Index API](#Index-API)
    * [JSON](#JSON)
    * [API Documentation](#API-Documentation)
    * [Python Dictionary](#Python-Dictionary)
    * [Routing API](#Routing-API)
        * [Scraping Useful Chicago Data](#Scraping-Useful-Chicago-Data)
        * [Geocoding](#Geocoding)
        * [Planner Resource Syntax](#Planner-Resource-Syntax)
        * [Incorporating Databases](#Incorporating-Databases)
    * [Isochrone API](#Isochrone-API)

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

In this module, we introduce two general ways that one can retrieve data from data sources on the Internet: APIs and web scraping. We've already covered web scraping, and given the messiness of that subject you may find yourself really appreciating the relative simplicity of using APIs - once you grasp the concepts.

API stands for "Application Programming Interface". An API is an agreed upon way for one computer program to interact with another computer program.  There are many different kinds of APIs.  Some facilitate interaction between computers over the Internet, some do not. In fact, the Python module SQL Alchemy that we used yesterday is a type of API for talking to databases - you'll see more on APIs for databases with SQL Alchemy later in this course.

For this session, we focus on web APIs over HTTP that let a user query and retrieve data over the Internet.  This type of API documents an agreed-upon structure and content of requests and responses a program can use to interact with a system.  As long as your code adheres to a system's API, it should be able to reliabily request and receive data from the system.

Below, we show how to make network API requests using HTTP(S).

## Learning Objectives

- Back to [Table of Contents](#Table-of-Contents)

** Learning objectives:**

- **Become familiar with different types of APIs.** Includes GET- and POST- based HTTP APIs, different formats of request bodies for POST-based APIs (form inputs, arbitrary JSON and XML, and then formalized dialects of each like SOAP), and how to learn a given API.
- **Learn the tools used to interact with network-based APIs.** Understand the tools for talking directly with APIs over HTTP connection, introduce libraries that abstract the details of the API and present a simplified programmatic interface, and then understand how to choose a tool.

## Topics

- Back to [Table of Contents](#Table-of-Contents)

Outline of topics covered in this notebook:

- Making raw HTTP API requests
- Using pre-packaged API client libraries
- Practical considerations - Knowing API rules and coding to follow them, and performance
- Example: OpenTripPlanner

# Setup - Load Python packages

- back to [Table of Contents](#Table-of-Contents)

In [ ]:
## import Python packages ##
import time # to convert time as needed and report how long some functions take

# interacting with websites and web-APIs
import requests # easy way to interact with web sites and services
import json # read/write JavaScript Object Notation (JSON)
from bs4 import BeautifulSoup

# data manipulation
import pandas as pd # easy data manipulation
# import geopandas as gpd # geographic data manipulation
# from geopandas.tools import sjoin, overlay # spatial join and overlay functions
# from shapely.geometry import Point, LineString # to create lines from a list of points

# visualization
import matplotlib as mplib
import matplotlib.pyplot as plt # visualization package

# so images get plotted in the notebook
%matplotlib inline

In [ ]:
print("Package versions")
print("requests: {}".format(requests.__version__))
print("json: {}".format(json.__version__))
print("pandas: {}".format(pd.__version__))
# print("geopandas: {}".format(gpd.__version__)) # check that correct version of geopandas is installed, should be v0.2+
print("matplotlib: {}".format(mplib.__version__))

# Using APIs

- Back to the [Table of Contents](#Table-of-Contents)

API overview
+ In general: APIs ([Application Programming Interfaces](https://en.wikipedia.org/wiki/Application_programming_interface)) are "set[s] of subroutine definitions, protocols, and tools for building software and applications. A good API makes it easier to develop a program by providing all the building blocks, which are then put together by the programmer."
+ Here we're looking at a **web-API**, a specific type of API which makes it easier to interact with some aspect of a website. In this course, we'll be using APIs to gether data in an automated way - like grabbing a bunch of prior tweets from Twitter. More generally, APIs can also be used to interact with websites in any way the API is designed to. For instance, you can post and delete tweets with Twitter's API, too.

We're going to start getting our hands dirty with the various APIs of the OpenTrip Planner (OTP) software. 

### OpenTripPlanner
[OpenTripPlanner (OTP)](http://docs.opentripplanner.org/en/latest/) is an open source routing software that provides a number of services, here we'll explore:
1. [Index API](#Index-API) - provides information about the data loaded into OTP, for instance what transportation agencies' data are included;
2. [Routing API](#Routing-API) - creates a plan for how to get from one location to another, with a number of additional options such as:
  * Departure time (and date) - if you're curious about a specific departure time or date;
  * transit modes - default is to consider any public transportation option in the system, but it can also be set to "AUTO" to do vehicle routing or "WALK" for walking only directions.
  
3. [Isochrone API](#Isochrone-API) - generates a polygon representing the area a traveler can reach if they start from a given location and travel for a specified amount of time (isochrone means 'equal time').

## RESTful APIs

- Back to the [Table of Contents](#Table-of-Contents)

The OTP APIs are what is called "[RESTful](https://en.wikipedia.org/wiki/Representational_state_transfer)" web services. REST stands for REpresentational State Transfer, but don't worry about the acronym so much as the idea. RESTful services adhere to a [series of requirements](https://www.restapitutorial/whatisrest.html) that enable them to be consistent, scalable, reliable, and relatively simple. RESTful APIs allow you to access a pre-defined set of operations through HTTP(S) requests. REST is fantastic because, in part, if you can generate the right URL, you'll always get the right response (this was not always the case with SOAP - the predecessor to REST).

To use a RESTful API, we'll need to understand (1) how to properly format the request and (2) how to manage and make use of the response from the API. Below we will walk through these concepts while using some of OTP's web services.

In [ ]:
## check what routers are installed on system 
# the below steps will be revisited and discussed further below

# base URL where OTP is installed - ennd point "routers/" simply lists information about the router
base_url = "https://tripplanner.adrf.info/otp/routers"

# get query response from API
response = requests.get(base_url)

# Convert response to text
response = json.loads(response.text)

# print routerId for each router returned
for router in response[u'routerInfo']:
    print(router['routerId'])

In [ ]:
### First, we need to set a few parameters. ###

# Router ID 
# - OTP can have many different routers available for different cities or subsets of transportation agencies.
# router_chicago = 'chicago_20180424' # Chicago  metro area transit router

router_kcmo  = 'kcmo_20180424' # KCMO router

# update base URL to add the routerID for the city you want to explore - 
# note the below code is based on the "chicago_20161028" router, if you select a different router other inputs
# would need to change
base_url = base_url+"/{}/".format(router_kcmo)
print(base_url)

### Index API

- Back to the [Table of Contents](#Table-of-Contents)

The Index API provides access to general information about the data loaded into a given OTP router (as specified by the 'routerID' variable set above). Full list of [options are here](http://dev.opentripplanner.org/apidoc/1.0.0/resource_IndexAPI.html).

Below, we can make a request simply by taking our base URL and adding the `feeds` endpoint. Here, we use the term endpoint to refer to the completed URL that links to the most granular aspect of an API. The combination of the router id, index API, and feeds request make up our endpoint. This will provide a list of data feeds available for the router we've selected.

In [ ]:
# Set up query URL
qry_url = '{}index/feeds'.format(base_url)

# Again, since we are still using HTTP, we can use the requests package's get 
response = requests.get(qry_url)

# Convert response to text
response = response.text

# Our response is a JSON array:
print(response)

In [ ]:
print(qry_url)

## JSON

- Back to the [Table of Contents](#Table-of-Contents)

[JSON](www.json.org) is a common non-tabular data format often used by services and software on the internet. We'll introduce JSON slowly, but it's helpful to know that it is oriented around the idea of `key-value` pairs. The `keys` refer to information about the data while the `values` is the data itself. For instance, if you were to translate a tabular dataset into JSON, the column names (and possibly row names/numbers) would become `keys`, while the data in the cells would become `values`. Our first JSON response is a simple array, the equivalent of a Python list.

In [ ]:
# Convert text to a Python object using the 'json' package
feeds = json.loads(response)

# And now we have a Python list:
print(type(feeds))
print(feeds)

This is just a list of feed IDs created by OTP - so there are three agencies providing data feeds to our version of the OTP. This is not particularly informative, but we did get a response from the web API. Let's check the 'agencies' endpoint to see what more it provides.

In [ ]:
## We can use the /agences resource of the Index API to get more information.
## Below, we combine the previous steps into one line and ask for the agency associated with the first feed:
print('{}index/agencies/{}'.format(base_url, '1')) 
print(json.loads(requests.get('{}index/agencies/{}'.format(base_url, '1')).text))

This is more helpful - we now know the agency associated with the feed id, as well as its website and other information. Let's use a loop to repeat this for the other feeds.

In [ ]:
## Let's do the same for each feed:
for feed in feeds:

    # print out which feed we're looking at on this pass of the loop
    print("feed {}".format(feed))
    
    # get agency information for this feed just as we did above, but using the feed from our list of feeds
    agency = json.loads(requests.get('{}index/agencies/{}'.format(base_url, feed)).text)
    
    print(agency)
        
    # add a blank line after each feed for legibility
    print('')


### API Documentation

- Back to the [Table of Contents](#Table-of-Contents)

If you want more information about what routes are included in a given feed, you can query the 'routes' resources as below. At this point, you may be wondering how you would know what resources and endpoints are available for a given API. This is where API documentation comes in. The [OpenTripPlanner Index API documentation](dev.opentripplanner.org/apidoc/0.20.0/resource_indexAPI.html) includes a list of valid HTTP methods (mostly get and a few post) for the resources and specific endpoints within the Index API.

For instance, there is a valid HTTP get request for the URL `/routers/{routerid}/index/agencies/{feedId}` where routerid and feedId are changeable parameters. This is how you would have known that the above requests would be successful.

Below, we can examine all the routes of one agency in the format: `/routers/{router_id}/index/agencies/{feedID}/{agencyID}/routes`

In [ ]:
# Using agency 'METRA' from first feed
qry_url = '{0}index/agencies/{1}/{2}/routes'.format(base_url, '1', '1')
print(qry_url)
routes = json.loads(requests.get(qry_url).text)
# print(routes)

In [ ]:
# Routes is a list - a data structure we are familiar with:
print(type(routes))

# However, the objects that makes up this list may be new to you, the python dictionary:
print(type(routes[0]))
print(type(routes[1]))
print(type(routes[2]))

## Python Dictionary

- Back to the [Table of Contents](#Table-of-Contents)

The [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), or dict for short, is a common type of python object used to store sets of key-value pairs. Sound familiar? It should! Dictionaries are python's internal counterpart to JSON data.

Here, we'll learn to grab data from within a dict by using the key name, following this syntax: `dict['key']`

In [ ]:
test_route = routes[3]
print(test_route)
print('------')
print(test_route['agencyName'])

In [ ]:
## Simple enough - let's use a loop and our new understanding of dicts to print out the mode and route name:
for route in routes:
    print('mode: {} | id: {} | route name: {}'.format(route['mode'], route['id'], route['longName']))

In [ ]:
## You can use the Index API to query all the stops along a route in the format:
## /otp/routers/{routerID}/index/routes/{routeID}/stops

## Remember the base url:
print(base_url)

## Can you query the stops along one of the Chicago METRA routes?



## Routing API

- Back to the [Table of Contents](#Table-of-Contents)

Now that we've tested we can access OTP from Jupyter, let's do something a bit more interesting: get a route plan between some locations. This will allow us to answer "How long will it take to get from *here* to *there*?"

Similar to the Index API, the [Routing API documentation](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html) tells us what features are available and how to access those feastures.

## Input Data

- Back to the [Table of Contents](#Table-of-Contents)

OTP works best with latitude and longitude coordinates. Here we are pulling a sample of the center points from KCMO census blocks and the center points of KCMO parcels to use as our origin and destination locations

In [ ]:
# Database connection
from sqlalchemy import create_engine
engine = create_engine("postgresql://stuffed/appliedda")

# get dataframes
query_kcmo_blocks = """
SELECT geoid10 block_fips, ST_X(ST_Centroid(geom_wgs)) lon, ST_Y(ST_Centroid(geom_wgs)) lat
FROM public.kcmo_blocks
LIMIT 100
"""
kcmo_blocks = pd.read_sql(query_kcmo_blocks, engine)

print(kcmo_blocks.head(1))

query_kcmo_parcels = """
SELECT gid, ST_X(ST_Centroid(geom)) lon, ST_Y(ST_Centroid(geom)) lat
FROM kansas_city_mo_parcels
LIMIT 100
"""
kcmo_parcels = pd.read_sql(query_kcmo_parcels, engine)

print(kcmo_parcels.head(1))

## Planner Resource Syntax

- Back to the [Table of Contents](#Table-of-Contents)

The Planner Resource API does trip planning based on a large number of customizable parameters. To give you a sense of all the options available, OTP's planner resource allows users to set the additional time it will take baord a vehicle (like a bus) with a bike, as opposed to boarding on foot. There are a lot of available options. This is great for us, since once we understand the simple syntax of this API, we can avail ourselves of this granular customization if we want to. 

This resource is located at `/OTP/routers/{routerID}/plan` and when setting options within a URL, they follow a single question mark. So first, let's plan a trip with just the required options, `fromPlace`, `toPlace`, and `datae` (which takes an option in the form YYY-MM-DD) which you can see we set after `/plan?` and separated by ampersands `&`:

In [ ]:
origin_lat = kcmo_parcels["lat"][2]
origin_lon = kcmo_parcels["lon"][2]

destination_lat = kcmo_blocks["lat"][10]
destination_lon = kcmo_blocks["lon"][10]

print(origin_lat, origin_lon, destination_lat, destination_lon)


In [ ]:

qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2018-04-23'\
.format(base_url, origin_lat, origin_lon, destination_lat, destination_lon)   
print(qry_url)

response = requests.get(qry_url)
response = response.text
plan = json.loads(response)

# Examine the response, which is a routing plan:
print(plan)

In [ ]:
# So again our JSON object was transformed into a Python dict.
print(type(plan))

In [ ]:
# We can look at the available keys:
print(plan.keys())

In [ ]:
# And use those keys to see the dict'svalues.
# For instance, let's print out the requestParameters:
print(plan['requestParameters'])

In [ ]:
# Dicts can contain other dicts, like in the case of the plan:
print(type(plan["plan"]))
print(plan['plan'].keys())

In [ ]:
## We can use a similar syntax torfer to the keys of a dict within a dict:
print(plan["plan"]["to"])

In [ ]:
# Time is stored in a raw computer format
print('raw time value: {}'.format(plan['plan']['date']))

# But we can convert it to a datetime object so it's comprehensible.
# note OTP returns raw time value with three extra zeros, divide by 1000 to get rid of them
print('datetime formatted: {}'.format(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(plan['plan']['date']/1000))))

In [ ]:
# 'itineraries' holds a lot more information, let's start with how many itineraries were returned
print(len(plan['plan']['itineraries']))

# and list what keys exit for the first itinerary
print(plan['plan']['itineraries'][0].keys())

In [ ]:
# compare the three itineraries across some pieces
for i in plan['plan']['itineraries']:
    print('duration (minutes) = {0:.2f} | transfers = {1:} | walkDist = {2:.2f} | \
legs = {4:} | startTime = {5:} | endTime = {6:}'\
.format(i['duration']/60., i['transfers'], i['walkDistance'], i['walkTime'], len(i['legs']), 
        time.strftime('%H:%M:%S', time.localtime(i['startTime']/1000)), 
        time.strftime('%H:%M:%S', time.localtime(i['endTime']/1000))))

In [ ]:
# note we just counted the length of the "legs" output, it contains the details of the actual route
# here is what is included in a "leg"
print(plan['plan']['itineraries'][0]['legs'][0].keys())

In [ ]:
# let's compare the three legs of the first itinerary, similarly as we compared the itineraries
for leg in plan['plan']['itineraries'][0]['legs']:
    print('distance = {:,.2f} | duration = {:.0f} | mode = {} | route = {} | steps = {}'.\
format(leg['distance'], leg['duration'], leg['mode'], leg['route'], len(leg['steps'])))

So, if mode is 'WALK' then route is blank and steps is a list. what is included in one of those 'steps'?

In [ ]:
print(plan['plan']['itineraries'][0]['legs'][0]['steps'][0].keys())

In [ ]:
# so, what streets does this first route call for a person to walk on?
for leg in plan['plan']['itineraries'][0]['legs']:
    print('leg sends person on following streets:')
    if leg['mode']=='WALK':
        for step in leg['steps']:
            print(step['streetName'])
    else:
        print('N/A - not a walking leg.')

## Adding Further Options:

The arriveBy parameter takes a time hh:mm:ss - can you add this option to our query?


In [ ]:
your_qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2018-04-23'\
.format(base_url, origin_lat, origin_lon, destination_lat, destination_lon)   
print(your_qry_url)

response = requests.get(your_qry_url)
response = response.text
plan = json.loads(response)

print(plan)

## Isochrone API

- Back to the [Table of Contents](#Table-of-Contents)

The Isochrone (meaning same-time) tool gives the area (as a polygon) a traveler can reach from a specified point within a travel time. Like the other APIs, the Isochrone API has many other query parameters the user can set if so desired, [description here](http://dev.opentripplanner.org/apidoc/1.0.0/resource_LIsochrone.html). It requires that we define a starting location, a mode of transportation, a date, and an amount of travel time.

Below, we start in downtown Chicago, allowing use of foot and public transit, on a certain date, and with 30 minutes of travel time allowed.

In [ ]:
# set start location
start_point = [origin_lat, origin_lon] # use our origin from above

travel_time = 60 * 30 # time in seconds, so this is 30 minutes
mode = "WALK,TRANSIT"

url = ("{}isochrone?fromPlace={},{}&mode={}&date=2018-04-23&cutoffSec={}").format(
    base_url,start_point[0],start_point[1],mode,travel_time)
print(url)

iso_response = requests.get(url)
print(iso_response.text)

In [ ]:
import geopandas as gpd

iso_json = json.loads(iso_response.text)

## load isochrone into a geopandas dataframe
iso_gdf = gpd.GeoDataFrame.from_features(iso_json['features'])
iso_gdf[:]

In [ ]:
# view the resulting isochrone shape (can you guess why there are separated geographies?)
iso_gdf.plot();

One potential use case for this functionality: can people at two locations reach some common location within a specified travel time?

In [ ]:
# 2nd location
start_point_2 = [destination_lat, destination_lon] # destination from above

url_2 = ("{}isochrone?fromPlace={},{}&mode={}&date=2018-04-23&cutoffSec={}").format(
    base_url,start_point_2[0],start_point[1],mode,travel_time)

# get json request
iso_json_2 = json.loads(requests.get(url_2).text)

## load isochrone into a geopandas dataframe
iso_gdf_2 = gpd.GeoDataFrame.from_features(iso_json_2['features'])

In [ ]:
# view the second isochrone
iso_gdf_2.plot();

In [ ]:
# do the two isochrones intersect?
iso_gdf.intersects(iso_gdf_2)

>> BELOW NOT UPDATED AS OF April 25

In [ ]:
# they do not intersect, does an overlay with a 'union' work anyway
# iso_join = gpd.overlay(iso_gdf, iso_gdf_2, how='union')

In [ ]:
# what does the dataframe look like now?
iso_join.head()

In [ ]:
# and visually?
iso_join.plot();

- Back to the [Table of Contents](#Table-of-Contents)

> A bit annoyingly this is difficult to tell where the two overlap. To fix this we can group based on the "time" and "time_2" columns to end with just 3 combinations: 
1. accessible from our first location only, 
2. accessible our second location only, and 
3. accessible from either location

We'll do this by using the ["dissolve" function](http://geopandas.org/aggregation_with_dissolve.html) from geopandas. However first we need to replace the "NaN" so those rows are not ignored

In [ ]:
# replace NaN with placeholder value, let's say 99999
iso_join.fillna(99999, inplace=True)

In [ ]:
iso_join = iso_join.dissolve(by=['time', 'time_2']).reset_index()

# Note: used reset_index() here so it's easier to use the 'time' and 'time_2' columns if needed

In [ ]:
# now what does it look like?
iso_join

In [ ]:
# and visually?
iso_join.plot();

In [ ]:
# add a label column to use so we can include a legend
iso_join['label'] = ''

# use index slicing function '.loc' of dataframes to update each value of label appropriately
iso_join.loc[0,'label'] = 'Both'
iso_join.loc[1,'label'] = 'point 1 only'
iso_join.loc[2,'label'] = 'point 2 only'

In [ ]:
# set up a nicer visualization with labels
f,ax = plt.subplots(figsize=(8,8))

# use geopandas to specify label column and adding a legend to the matplotlib object 'ax'
iso_join.plot(column='label', ax=ax, legend=True);

# also plot start and stop points on the same map, note matplotlib takes [x,y] coordinates
ax.plot(start_point[1], start_point[0], 's', color='orange', markersize=10, label='point 1')
ax.plot(start_point_2[1], start_point_2[0], 's', color='grey', markersize=10, label='point 2')

# add some other labels
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# title
ax.set_title('Area accessible within {} minutes travel using public transit'.format(travel_time/60));